# Quantitative Momentum Strategy

"Momentum investing": investing in the stocks that have increased in price the most.

We will build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library imports

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Import our list of stocks

* refer to prj1 jupyternotebook

In [2]:
stocks_full = pd.read_csv('sp_500_stocks.csv')
stocks = pd.DataFrame({'ticker': stocks_full['Symbol']})
from secrets import IEX_CLOUD_API_TOKEN

## Make our first API call

Make the momentum screener - we need to get one-year price returns for each stock in the universe.

In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2302905934099,
 'week52high': 137.59,
 'week52low': 56.31,
 'week52change': 0.8782387346930561,
 'sharesOutstanding': 17219878073,
 'float': 0,
 'avg10Volume': 118124132,
 'avg30Volume': 107530988,
 'day200MovingAvg': 114.24,
 'day50MovingAvg': 124.68,
 'employees': 0,
 'ttmEPS': 3.28,
 'ttmDividendRate': 0.8368141342576241,
 'dividendYield': 0.006205626261380533,
 'nextDividendDate': '0',
 'exDividendDate': '2020-11-04',
 'nextEarningsDate': '0',
 'peRatio': 40.96753626999841,
 'beta': 1.160222613647073,
 'maxChangePercent': 51.97817872139487,
 'year5ChangePercent': 4.529375640823101,
 'year2ChangePercent': 2.6583928719159,
 'year1ChangePercent': 0.8921761892465431,
 'ytdChangePercent': 0.8919406098137824,
 'month6ChangePercent': 0.4966704740098275,
 'month3ChangePercent': 0.18638247137999314,
 'month1ChangePercent': 0.163716704365052,
 'day30ChangePercent': 0.157793275355081,
 'day5ChangePercent': 0.02335003677173134}

## Parse API call

This API call has all the information we need. We can parse it following the example below:

In [6]:
data['year1ChangePercent']

0.8921761892465431

## Execute a batch API call & build our DataFrame

In [9]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:(i+n)]


symbol_groups = list(chunks(stocks['ticker'], 100))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

    
colns = ['Ticker', 'Stock Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns = colns)

In [11]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                ],
                index = colns,
            ),
            ignore_index = True
        )

final_df

,Ticker,Stock Price,One-Year Price Return,Number of Shares to Buy
0,MMM,178.950,0.0175453,N/A
1,AOS,56.090,0.168356,N/A
2,ABT,111.930,0.261616,N/A
3,ABBV,108.750,0.249108,N/A
4,ABMD,331.615,0.899678,N/A
5,ACN,270.230,0.235773,N/A
6,ATVI,94.551,0.56199,N/A
7,ADBE,507.990,0.520233,N/A
8,AAP,166.100,-0.00276636,N/A
9,AMD,94.759,0.995335,N/A


## Remove low-momentum stocks

The investment strategy we are building seeks to identify the 50 highest-momentum stocks in the S&P 500.
So we will remove all the stocks in our DataFrame that fall below this momentum threshold by sorting the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

In [15]:
final_df.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_df = final_df.head(50)
final_df.reset_index(inplace=True)

final_df

/Users/bingjieouyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,index,Ticker,Stock Price,One-Year Price Return,Number of Shares to Buy
0,87,CARR,38.400,2.1175,N/A
1,346,NVDA,547.780,1.22397,N/A
2,360,PYPL,238.200,1.13815,N/A
3,16,ALB,156.830,1.11197,N/A
4,277,LB,39.370,1.09749,N/A
5,9,AMD,94.759,0.995335,N/A
6,81,CDNS,140.538,0.940065,N/A
7,203,FCX,26.570,0.914754,N/A
8,409,NOW,562.290,0.907225,N/A
9,46,AAPL,134.410,0.905887,N/A


## Calculate the number of shares to buy

In [17]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')
    
    try:
        float(portfolio_size)
    except:
        print('That is not a number! \nPlease try again.')
        portfolio_size = input('Enter the size of your portfolio: ')

In [18]:
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 1000000
1000000


In [22]:
position_size = float(portfolio_size)/len(final_df)

for i in range(0, len(final_df)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Stock Price'])

final_df

,index,Ticker,Stock Price,One-Year Price Return,Number of Shares to Buy
0,87,CARR,38.400,2.1175,520
1,346,NVDA,547.780,1.22397,36
2,360,PYPL,238.200,1.13815,83
3,16,ALB,156.830,1.11197,127
4,277,LB,39.370,1.09749,508
5,9,AMD,94.759,0.995335,211
6,81,CDNS,140.538,0.940065,142
7,203,FCX,26.570,0.914754,752
8,409,NOW,562.290,0.907225,35
9,46,AAPL,134.410,0.905887,148


## Build a better (and more realistic) momentum strategy

* "High quality" momentum stocks: show "slow and steady" outperformance over long periods of time.
* "Low quality" momentum stocks: may not show any momentum for a long time, and then surge upwards.

To identify "high quality" momentum stocks, we will build a strategy that selects stocks from the highest percentile in 1-month, 3-month, 6-month, and 1-year price returns.

In [53]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
]

hqm_df = pd.DataFrame(columns = hqm_columns)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile


In [54]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A'
                ],
                index = hqm_columns,
            ),
            ignore_index = True
        )
        
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,178.330,N/A,0.0179818,N/A,0.138695,N/A,0.100932,N/A,-0.0173973,N/A
1,AOS,56.340,N/A,0.176386,N/A,0.189952,N/A,0.0240566,N/A,-0.0417831,N/A
2,ABT,112.520,N/A,0.261869,N/A,0.234762,N/A,0.0347005,N/A,0.00682294,N/A
3,ABBV,109.242,N/A,0.243861,N/A,0.114853,N/A,0.226277,N/A,-0.00185807,N/A
4,ABMD,322.180,N/A,0.911817,N/A,0.337592,N/A,0.18092,N/A,0.187092,N/A
5,ACN,267.870,N/A,0.23146,N/A,0.224049,N/A,0.157264,N/A,0.0311539,N/A
6,ATVI,94.950,N/A,0.579078,N/A,0.21443,N/A,0.134994,N/A,0.176086,N/A
7,ADBE,521.940,N/A,0.541827,N/A,0.187837,N/A,0.0274165,N/A,0.0544039,N/A
8,AAP,160.870,N/A,-0.00289514,N/A,0.103682,N/A,0.0350748,N/A,0.0480067,N/A
9,AMD,93.192,N/A,0.970449,N/A,0.814305,N/A,0.111283,N/A,0.0401886,N/A


## Calculate momentum percentiles

We will now calculate the momentum percentile scores for every stock in the 505 stocks.

In [69]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for time_period in time_periods:
    hqm_df.loc[hqm_df[f'{time_period} Price Return'].isna(), f'{time_period} Price Return'] = 0.0

for row in hqm_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_df.loc[row, percentile_col] = stats.percentileofscore(hqm_df[change_col], hqm_df.loc[row, change_col])/100 # divide 100 for formatting later in xlsx
        
hqm_df

/Users/bingjieouyang/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,19,ALGN,541.500,36,0.891474,0.86,0.982172,0.96,0.63783,0.94,0.0971624,0.64,96.7822
1,16,ALB,153.800,130,1.0726,0.98,0.946386,0.94,0.687968,0.96,0.0670142,0.36,95.0495
2,203,FCX,26.845,745,0.907183,0.9,1.25361,0.98,0.623543,0.9,0.0544637,0.22,93.2673
3,422,SIVB,395.930,50,0.514575,0.62,0.84261,0.86,0.596013,0.84,0.0633254,0.32,92.0792
4,359,PAYC,467.690,42,0.688912,0.76,0.499179,0.42,0.496602,0.8,0.0772292,0.4,90.8911
5,451,TWTR,55.996,357,0.677849,0.74,0.854245,0.88,0.224092,0.16,0.168546,0.88,88.9109
6,443,DIS,183.180,109,0.226678,0.3,0.607622,0.66,0.429819,0.68,0.205607,0.94,88.6634
7,18,ALXN,158.681,126,0.440021,0.56,0.403229,0.18,0.400446,0.58,0.254868,0.98,88.5644
8,240,IDXX,504.174,39,0.903826,0.88,0.531444,0.54,0.289268,0.32,0.0909631,0.44,88.2673
9,81,CDNS,138.270,144,0.936829,0.96,0.426355,0.24,0.279614,0.28,0.150512,0.82,87.4752


## Calculate the HQM (High quality momentum) score

HQM score = the mean of the 4 momentum percentile scores we calculated in the last section.

In [70]:
from statistics import mean

hqm_df['HQM Score'] = 'N/A'

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_df

/Users/bingjieouyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/bingjieouyang/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,19,ALGN,541.500,36,0.891474,0.86,0.982172,0.96,0.63783,0.94,0.0971624,0.64,0.85
1,16,ALB,153.800,130,1.0726,0.98,0.946386,0.94,0.687968,0.96,0.0670142,0.36,0.81
2,203,FCX,26.845,745,0.907183,0.9,1.25361,0.98,0.623543,0.9,0.0544637,0.22,0.75
3,422,SIVB,395.930,50,0.514575,0.62,0.84261,0.86,0.596013,0.84,0.0633254,0.32,0.66
4,359,PAYC,467.690,42,0.688912,0.76,0.499179,0.42,0.496602,0.8,0.0772292,0.4,0.595
5,451,TWTR,55.996,357,0.677849,0.74,0.854245,0.88,0.224092,0.16,0.168546,0.88,0.665
6,443,DIS,183.180,109,0.226678,0.3,0.607622,0.66,0.429819,0.68,0.205607,0.94,0.645
7,18,ALXN,158.681,126,0.440021,0.56,0.403229,0.18,0.400446,0.58,0.254868,0.98,0.575
8,240,IDXX,504.174,39,0.903826,0.88,0.531444,0.54,0.289268,0.32,0.0909631,0.44,0.545
9,81,CDNS,138.270,144,0.936829,0.96,0.426355,0.24,0.279614,0.28,0.150512,0.82,0.575


## Select the 50 best momentum stocks

In [71]:
hqm_df.sort_values('HQM Score', ascending=False, inplace=True)
hqm_df.reset_index(inplace=True, drop=True)

hqm_df = hqm_df[:50]
hqm_df

/Users/bingjieouyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,19,ALGN,541.500,36,0.891474,0.86,0.982172,0.96,0.63783,0.94,0.0971624,0.64,0.85
1,16,ALB,153.800,130,1.0726,0.98,0.946386,0.94,0.687968,0.96,0.0670142,0.36,0.81
2,203,FCX,26.845,745,0.907183,0.9,1.25361,0.98,0.623543,0.9,0.0544637,0.22,0.75
3,274,KSS,42.712,468,-0.191812,0.02,0.935184,0.9,1.17951,1,0.198321,0.92,0.71
4,451,TWTR,55.996,357,0.677849,0.74,0.854245,0.88,0.224092,0.16,0.168546,0.88,0.665
5,422,SIVB,395.930,50,0.514575,0.62,0.84261,0.86,0.596013,0.84,0.0633254,0.32,0.66
6,382,PVH,97.830,204,-0.103022,0.06,0.945013,0.92,0.607262,0.88,0.11947,0.76,0.655
7,443,DIS,183.180,109,0.226678,0.3,0.607622,0.66,0.429819,0.68,0.205607,0.94,0.645
8,234,HWM,29.660,674,-0.104157,0.04,0.741526,0.8,0.624588,0.92,0.141275,0.8,0.64
9,143,DFS,91.000,219,0.0700298,0.12,0.840097,0.82,0.599619,0.86,0.116222,0.72,0.63


## Calculate the number of shares to buy

In [72]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [73]:
position_size = float(portfolio_size)/len(hqm_df)

for i in hqm_df.index:
    hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_df.loc[i, 'Price'])
    
hqm_df

,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,19,ALGN,541.500,36,0.891474,0.86,0.982172,0.96,0.63783,0.94,0.0971624,0.64,0.85
1,16,ALB,153.800,130,1.0726,0.98,0.946386,0.94,0.687968,0.96,0.0670142,0.36,0.81
2,203,FCX,26.845,745,0.907183,0.9,1.25361,0.98,0.623543,0.9,0.0544637,0.22,0.75
3,274,KSS,42.712,468,-0.191812,0.02,0.935184,0.9,1.17951,1,0.198321,0.92,0.71
4,451,TWTR,55.996,357,0.677849,0.74,0.854245,0.88,0.224092,0.16,0.168546,0.88,0.665
5,422,SIVB,395.930,50,0.514575,0.62,0.84261,0.86,0.596013,0.84,0.0633254,0.32,0.66
6,382,PVH,97.830,204,-0.103022,0.06,0.945013,0.92,0.607262,0.88,0.11947,0.76,0.655
7,443,DIS,183.180,109,0.226678,0.3,0.607622,0.66,0.429819,0.68,0.205607,0.94,0.645
8,234,HWM,29.660,674,-0.104157,0.04,0.741526,0.8,0.624588,0.92,0.141275,0.8,0.64
9,143,DFS,91.000,219,0.0700298,0.12,0.840097,0.82,0.599619,0.86,0.116222,0.72,0.63


## Formatting Excel output

In [74]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_df.to_excel(writer, sheet_name = 'momentum', index=False)

In [75]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_format = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

float_format = writer.book.add_format(
    {
        'num_format': '0.0000',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [76]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format],
    'D': ['One-Year Price Return', percent_format],
    'E': ['One-Year Return Percentile', percent_format],
    'F': ['Six-Month Price Return', percent_format],
    'G': ['Six-Month Return Percentile', percent_format],
    'H': ['Three-Month Price Return', percent_format],
    'I': ['Three-Month Return Percentile', percent_format],
    'J': ['One-Month Price Return', percent_format],
    'K': ['One-Month Return Percentile', percent_format],
    'L': ['HQM Score', integer_format],
}

for column in column_formats.keys():
    writer.sheets['momentum'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['momentum'].write(f'{column}1', column_formats[column][0], string_format)

writer.save()